# OmniQ Configuration-Based Usage Examples

This notebook demonstrates the various ways to configure OmniQ components using different configuration methods. OmniQ provides flexible configuration options to suit different use cases and deployment scenarios.

## Configuration Methods Covered

1. **Config Objects**: Type-validated configuration using `msgspec.Struct` classes
2. **Dictionary Configuration**: Simple dictionary-based configuration
3. **YAML File Configuration**: External configuration files
4. **Environment Variable Overrides**: Runtime configuration
5. **Mixed Configuration**: Combining different methods

Let's start by importing the necessary modules and setting up our environment.

In [ ]:
import os
import time
import datetime as dt
from pathlib import Path
import tempfile
import shutil

# Import OmniQ components
from omniq import OmniQ
from omniq.models import FileTaskQueueConfig, SQLiteResultStorageConfig
from omniq.queue import FileTaskQueue
from omniq.storage import SQLiteResultStorage

print("✓ Imported OmniQ components successfully")

## Define Example Task

Let's define a simple task function that we'll use throughout our examples:

In [ ]:
def example_task(name: str, multiplier: int = 1) -> str:
    """Example task function for demonstration."""
    result = f"Hello {name}!" * multiplier
    print(f"Task executed: {result}")
    return result

print("✓ Defined example task function")

## 1. Config Objects (Type-Validated Configuration)

Config objects provide type validation and IDE support. They use `msgspec.Struct` classes to ensure configuration correctness at runtime.

In [ ]:
print("=== Config Objects Example ===")

# Create components using specific config classes
queue_config = FileTaskQueueConfig(
    project_name="config_objects_demo",
    base_dir="./temp/config_objects",
    queues=["high", "medium", "low"]
)

result_config = SQLiteResultStorageConfig(
    project_name="config_objects_demo",
    base_dir="./temp/config_objects"
)

# Create components from config objects
queue = FileTaskQueue.from_config(queue_config)
result_store = SQLiteResultStorage.from_config(result_config)

# Create OmniQ instance
oq = OmniQ(
    project_name="config_objects_demo",
    task_queue=queue,
    result_store=result_store
)

print("✓ Created OmniQ with config objects")
print(f"  Queue type: {type(queue).__name__}")
print(f"  Result store type: {type(result_store).__name__}")

In [ ]:
# Test the config objects setup
with oq:
    oq.start_worker()
    
    # Enqueue a task
    task_id = oq.enqueue(
        func=example_task,
        func_args={"name": "Config Objects", "multiplier": 2},
        queue_name="high"
    )
    
    print(f"✓ Enqueued task: {task_id}")
    
    # Wait for task completion
    time.sleep(2)
    
    # Get result
    result = oq.get_result(task_id)
    print(f"✓ Task result: {result}")
    
    oq.stop_worker()

## 2. Dictionary Configuration

Dictionary configuration provides a simple and flexible way to configure OmniQ components using Python dictionaries.

In [ ]:
print("=== Dictionary Configuration Example ===")

# Define configuration as dictionary
config_dict = {
    "project_name": "dict_config_demo",
    "task_queue": {
        "type": "file",
        "config": {
            "base_dir": "./temp/dict_config",
            "queues": ["high", "medium", "low"]
        }
    },
    "result_store": {
        "type": "sqlite",
        "config": {
            "base_dir": "./temp/dict_config"
        }
    },
    "worker": {
        "type": "thread_pool",
        "config": {
            "max_workers": 5
        }
    }
}

# Create OmniQ from dictionary
oq_dict = OmniQ.from_dict(config_dict)
print("✓ Created OmniQ from dictionary config")
print(f"  Project name: {config_dict['project_name']}")
print(f"  Task queue type: {config_dict['task_queue']['type']}")
print(f"  Result store type: {config_dict['result_store']['type']}")
print(f"  Worker type: {config_dict['worker']['type']}")

In [ ]:
# Test the dictionary configuration
with oq_dict:
    oq_dict.start_worker()
    
    # Enqueue a task
    task_id = oq_dict.enqueue(
        func=example_task,
        func_args={"name": "Dictionary Config", "multiplier": 1},
        queue_name="medium"
    )
    
    print(f"✓ Enqueued task: {task_id}")
    
    # Wait for task completion
    time.sleep(2)
    
    # Get result
    result = oq_dict.get_result(task_id)
    print(f"✓ Task result: {result}")
    
    oq_dict.stop_worker()

## 3. YAML File Configuration

YAML configuration allows you to store configuration in external files, making it easy to manage different environments and deployments.

In [ ]:
print("=== YAML File Configuration Example ===")

# Create a temporary YAML config file
yaml_config_content = """
project_name: yaml_config_demo

task_queue:
  type: file
  config:
    base_dir: ./temp/yaml_config
    queues:
      - high
      - medium
      - low

result_store:
  type: sqlite
  config:
    base_dir: ./temp/yaml_config

worker:
  type: thread_pool
  config:
    max_workers: 8
"""

# Write the config to a temporary file
config_file_path = "./temp_config.yaml"
with open(config_file_path, 'w') as f:
    f.write(yaml_config_content)

print(f"✓ Created temporary YAML config file: {config_file_path}")

# Display the YAML content
print("\nYAML Configuration Content:")
print(yaml_config_content)

In [ ]:
# Load OmniQ configuration from YAML file
oq_yaml = OmniQ.from_config_file(config_file_path)
print("✓ Created OmniQ from YAML config file")

# Test the YAML configuration
with oq_yaml:
    oq_yaml.start_worker()
    
    # Enqueue a task
    task_id = oq_yaml.enqueue(
        func=example_task,
        func_args={"name": "YAML Config", "multiplier": 3},
        queue_name="low"
    )
    
    print(f"✓ Enqueued task: {task_id}")
    
    # Wait for task completion
    time.sleep(2)
    
    # Get result
    result = oq_yaml.get_result(task_id)
    print(f"✓ Task result: {result}")
    
    oq_yaml.stop_worker()

# Clean up the temporary config file
os.remove(config_file_path)
print(f"✓ Cleaned up temporary config file")

## 4. Environment Variable Overrides

Environment variables provide runtime configuration overrides, useful for deployment scenarios where configuration needs to be changed without modifying code.

In [ ]:
print("=== Environment Variable Overrides Example ===")

# Set environment variables (these would typically be set externally)
env_vars = {
    "OMNIQ_TASK_QUEUE_TYPE": "file",
    "OMNIQ_RESULT_STORAGE_TYPE": "sqlite",
    "OMNIQ_MAX_WORKERS": "12",
    "OMNIQ_TASK_QUEUE_URL": "./temp/env_config",
    "OMNIQ_RESULT_STORAGE_URL": "./temp/env_config"
}

# Set the environment variables
for key, value in env_vars.items():
    os.environ[key] = value

print("✓ Set environment variables:")
for key, value in env_vars.items():
    print(f"  {key}={value}")

# Create configuration that can be overridden by environment variables
base_config = {
    "project_name": "env_config_demo",
    "task_queue": {
        "type": "file",  # This could be overridden by OMNIQ_TASK_QUEUE_TYPE
        "config": {
            "base_dir": "./temp/env_config",
            "queues": ["high", "medium", "low"]
        }
    },
    "result_store": {
        "type": "sqlite",  # This could be overridden by OMNIQ_RESULT_STORAGE_TYPE
        "config": {
            "base_dir": "./temp/env_config"
        }
    }
}

oq_env = OmniQ.from_dict(base_config)
print("✓ Created OmniQ with environment variable overrides")

In [ ]:
# Test the environment variable configuration
with oq_env:
    oq_env.start_worker()
    
    # Enqueue a task
    task_id = oq_env.enqueue(
        func=example_task,
        func_args={"name": "Environment Config", "multiplier": 1},
        queue_name="high"
    )
    
    print(f"✓ Enqueued task: {task_id}")
    
    # Wait for task completion
    time.sleep(2)
    
    # Get result
    result = oq_env.get_result(task_id)
    print(f"✓ Task result: {result}")
    
    oq_env.stop_worker()

# Clean up environment variables
for key in env_vars.keys():
    if key in os.environ:
        del os.environ[key]

print("✓ Cleaned up environment variables")

## 5. Mixed Configuration Methods

You can combine different configuration methods to create flexible setups that suit your specific needs.

In [ ]:
print("=== Mixed Configuration Methods Example ===")

# Use config objects for task queue
queue_config = FileTaskQueueConfig(
    project_name="mixed_config_demo",
    base_dir="./temp/mixed_config",
    queues=["priority", "normal"]
)

queue = FileTaskQueue.from_config(queue_config)
print("✓ Created task queue from config object")

# Use dictionary for result storage
result_store_dict = {
    "project_name": "mixed_config_demo",
    "base_dir": "./temp/mixed_config"
}

result_store = SQLiteResultStorage.from_dict(result_store_dict)
print("✓ Created result storage from dictionary")

# Create OmniQ with mixed configuration
oq_mixed = OmniQ(
    project_name="mixed_config_demo",
    task_queue=queue,
    result_store=result_store
)

print("✓ Created OmniQ with mixed configuration methods")
print(f"  Task queue: Config object -> {type(queue).__name__}")
print(f"  Result store: Dictionary -> {type(result_store).__name__}")

In [ ]:
# Test the mixed configuration
with oq_mixed:
    oq_mixed.start_worker()
    
    # Enqueue a task
    task_id = oq_mixed.enqueue(
        func=example_task,
        func_args={"name": "Mixed Config", "multiplier": 2},
        queue_name="priority"
    )
    
    print(f"✓ Enqueued task: {task_id}")
    
    # Wait for task completion
    time.sleep(2)
    
    # Get result
    result = oq_mixed.get_result(task_id)
    print(f"✓ Task result: {result}")
    
    oq_mixed.stop_worker()

## Configuration Comparison

Let's compare the different configuration methods and their use cases:

In [ ]:
print("=== Configuration Methods Comparison ===")
print()

comparison_data = [
    {
        "Method": "Config Objects",
        "Type Safety": "✓ Strong",
        "IDE Support": "✓ Full",
        "Validation": "✓ Runtime",
        "External Files": "✗ No",
        "Best For": "Development, Type Safety"
    },
    {
        "Method": "Dictionary",
        "Type Safety": "✗ None",
        "IDE Support": "✗ Limited",
        "Validation": "✗ None",
        "External Files": "✗ No",
        "Best For": "Simple, Dynamic Config"
    },
    {
        "Method": "YAML Files",
        "Type Safety": "✗ None",
        "IDE Support": "~ Partial",
        "Validation": "~ Load-time",
        "External Files": "✓ Yes",
        "Best For": "Deployment, Environment Management"
    },
    {
        "Method": "Environment Variables",
        "Type Safety": "✗ None",
        "IDE Support": "✗ None",
        "Validation": "✗ None",
        "External Files": "✓ Yes",
        "Best For": "Runtime Overrides, Docker/K8s"
    },
    {
        "Method": "Mixed",
        "Type Safety": "~ Partial",
        "IDE Support": "~ Partial",
        "Validation": "~ Partial",
        "External Files": "~ Optional",
        "Best For": "Complex Scenarios, Flexibility"
    }
]

# Print comparison table
header = f"{'Method':<20} {'Type Safety':<12} {'IDE Support':<12} {'Validation':<12} {'External':<10} {'Best For':<30}"
print(header)
print("-" * len(header))

for item in comparison_data:
    row = f"{item['Method']:<20} {item['Type Safety']:<12} {item['IDE Support']:<12} {item['Validation']:<12} {item['External Files']:<10} {item['Best For']:<30}"
    print(row)

## Cleanup

Let's clean up the temporary files created during our examples:

In [ ]:
print("=== Cleanup ===")

# List of temporary directories to clean up
temp_dirs = [
    "./temp/config_objects",
    "./temp/dict_config",
    "./temp/yaml_config",
    "./temp/env_config",
    "./temp/mixed_config"
]

cleaned_count = 0
for temp_dir in temp_dirs:
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)
        print(f"✓ Cleaned up {temp_dir}")
        cleaned_count += 1
    else:
        print(f"- {temp_dir} (not found)")

print(f"\n✅ Cleanup completed! Removed {cleaned_count} temporary directories.")

## Summary

In this notebook, we've explored the various configuration methods available in OmniQ:

1. **Config Objects**: Provide type safety and IDE support, ideal for development
2. **Dictionary Configuration**: Simple and flexible, good for dynamic configurations
3. **YAML File Configuration**: External files for deployment and environment management
4. **Environment Variable Overrides**: Runtime configuration for containerized deployments
5. **Mixed Configuration**: Combining methods for maximum flexibility

Choose the configuration method that best fits your use case:

- **Development**: Use config objects for type safety and IDE support
- **Simple applications**: Use dictionary configuration for straightforward setups
- **Production deployments**: Use YAML files for environment-specific configurations
- **Containerized applications**: Use environment variables for runtime overrides
- **Complex scenarios**: Mix different methods as needed

The flexibility of OmniQ's configuration system allows you to adapt to different deployment scenarios and development workflows while maintaining clean, maintainable code.